In [1]:
from pymongo import MongoClient
# from sshtunnel import SSHTunnelForwarder
import pymongo
from tqdm import tqdm
import os
import re

import numpy as np
import pandas as pd

In [2]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
! nvidia-smi

Thu Jun 23 02:00:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 26%   34C    P8    21W / 260W |   1957MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 25%   

# MongoDB

In [4]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    db_de = db.patent_de
    db_us = db.patent_us
    db_cn = db.patent_cn
except errors.ConnectionFailure as err:
    print(err)

In [5]:
db_de

Collection(Database(MongoClient(host=['140.117.69.70:30241'], document_class=dict, tz_aware=False, connect=True, authmechanism='SCRAM-SHA-1'), 'eva'), 'patent_de')

## DE

In [6]:
data_assignee = db_de.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    #print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [7]:
len(ls_company)

446

In [8]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_de.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc_de = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc_de

,company,cpc,count
0,Intel Corp,G06F9,392
1,Intel Corp,H01L2224,239
2,Intel Corp,H04L67,157
3,Intel Corp,G06N3,153
4,Intel Corp,G06F12,134
...,...,...,...
14385,Trumpf Werkzeugmaschinen GmbH and Co KG,00,1
14386,Trumpf Werkzeugmaschinen GmbH and Co KG,1474,1
14387,Trumpf Werkzeugmaschinen GmbH and Co KG,H05K7,1
14388,Trumpf Werkzeugmaschinen GmbH and Co KG,043,1


In [9]:
df_cpc_de.shape

(14390, 3)

In [10]:
# 取cpc前3碼
df_cpc_de["cpc_n"] = df_cpc_de.cpc.apply(lambda x: x[0:3])

In [11]:
df_cpc_de

,company,cpc,count,cpc_n
0,Intel Corp,G06F9,392,G06
1,Intel Corp,H01L2224,239,H01
2,Intel Corp,H04L67,157,H04
3,Intel Corp,G06N3,153,G06
4,Intel Corp,G06F12,134,G06
...,...,...,...,...
14385,Trumpf Werkzeugmaschinen GmbH and Co KG,00,1,00
14386,Trumpf Werkzeugmaschinen GmbH and Co KG,1474,1,147
14387,Trumpf Werkzeugmaschinen GmbH and Co KG,H05K7,1,H05
14388,Trumpf Werkzeugmaschinen GmbH and Co KG,043,1,043


In [12]:
# 轉成company-cpc 的總數表格
table_de = df_cpc_de.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_de = table_de.droplevel(level=0, axis=1)
table_de = table_de.rename(columns={'':'company'})

In [13]:
# 取有英文的部份
c_ls_de = [c for c in table_de.columns if re.search('[A-Za-z]', c)]
final_table_de = table_de[c_ls_de]
final_table_de.head()

cpc_n,company,A01,A24,A41,A42,A43,A44,A45,A47,A61,...,G11,G16,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,ABB Schweiz AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
1,ACO Severin Ahlmann GmbH and Co KG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
2,ALBA GROUP PLC & Co KG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,AT&S Austria Technologie and Systemtechnik AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.0,0.0,0.0,0.0,102.0,0.0,0.0,0.0
4,AU Optronics Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
final_table_de.shape

(437, 84)

In [67]:
final_table_de.to_csv('company_cpc_de.csv',index=0)

In [14]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.company_cpc_de

In [15]:
# title
conn_db_collection.insert_one({"company":final_table_de.columns.tolist()[0],"cpc":final_table_de.columns.tolist()[1:]})

In [16]:
# 內容
for k in final_table_de.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"cpc":k[1:]})

## CN

In [52]:
data_assignee = db_cn.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [53]:
len(ls_company)

13002

In [18]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_cn.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc.head()

In [20]:
df_cpc.head()

,company,cpc,count
0,Qualcomm Inc,H04L5,5583
1,Qualcomm Inc,H04W72,4430
2,Qualcomm Inc,H04L1,2496
3,Qualcomm Inc,H04B7,2182
4,Qualcomm Inc,H04W52,1185


In [19]:
df_cpc.shape

(177888, 3)

In [21]:
# 取cpc前3碼
df_cpc["cpc_n"] = df_cpc.cpc.apply(lambda x: x[0:3])

In [22]:
# 轉成company-cpc 的總數表格
table_cn = df_cpc.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_cn = table_cn.droplevel(level=0, axis=1)
table_cn = table_cn.rename(columns={'':'company'})

In [23]:
# 取有英文的部份
c_ls_cn = [c for c in table_cn.columns if re.search('[A-Za-z]', c)]
final_table_cn = table_cn[c_ls_cn]
final_table_cn.head()

cpc_n,company,A01,A21,A23,A24,A41,A42,A43,A44,A45,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,(c-Dot),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0
1,01 Bulletin Laboratory Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
2,10757616 Canada Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,12.0,28.0,0.0,0.0,0.0,2.0,2.0,0.0
3,10757616加拿大公司,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,12.0,28.0,0.0,0.0,0.0,2.0,2.0,0.0
4,"11th Street Co., Ltd.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9.0,0.0,1.0,0.0,0.0


In [24]:
final_table_cn.shape

(8608, 124)

In [66]:
final_table_cn.to_csv('company_cpc_cn.csv',index=0)

In [25]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.company_cpc_cn

In [26]:
# title
conn_db_collection.insert_one({"company":final_table_cn.columns.tolist()[0],"cpc":final_table_cn.columns.tolist()[1:]})

In [27]:
# 內容
for k in final_table_cn.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"cpc":k[1:]})

## US

In [54]:
data_assignee = db_us.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}#,
    #{"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"].replace("))",")"))
    ls_count.append(data["count"])

In [55]:
len(ls_company)

5623

In [29]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    try:
        data_cpc = db_us.aggregate([
            {"$match" : {"current_assignee":{"$regex":c}}},
            {"$unwind" : "$current_assignee"},
            {"$unwind" : "$classification"},
            {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
            {"$unwind" : "$class"},
            {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
            {"$sort": {"count":-1}}
            # {"$limit":5}
        ])
    except:
        print(c)

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc_us = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc_us.head()

,company,cpc,count
0,Qualcomm Inc,H04L5,10569
1,Qualcomm Inc,H04W72,9022
2,Qualcomm Inc,H04L1,4813
3,Qualcomm Inc,H04B7,4541
4,Qualcomm Inc,H04W52,2422


In [30]:
df_cpc_us.shape

(225532, 3)

In [31]:
# 取cpc前3碼
df_cpc_us["cpc_n"] = df_cpc_us.cpc.apply(lambda x: x[0:3])

In [32]:
# 轉成company-cpc 的總數表格
table_us = df_cpc_us.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_us = table_us.droplevel(level=0, axis=1)
table_us = table_us.rename(columns={'':'company'})

In [34]:
# 取有英文的部份
c_ls_us = [c for c in table_us.columns if re.search('[A-Za-z]', c)]
final_table_us = table_us[c_ls_us]
final_table_us.head()

cpc_n,company,A01,A21,A22,A23,A24,A41,A42,A43,A44,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,#glowontech,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01 COMMUNIQUE LABORATORY Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0
2,0Chain LLC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
3,10353744 Canada Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
4,10757616 Canada Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.0,14.0,0.0,0.0,0.0,1.0,1.0,0.0


In [35]:
final_table_us.shape

(5587, 121)

In [65]:
final_table_us.to_csv('company_cpc_us.csv',index=0)

In [39]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.company_cpc_us

In [40]:
# title
conn_db_collection.insert_one({"company":final_table_us.columns.tolist()[0],"cpc":final_table_us.columns.tolist()[1:]})

In [41]:
# 內容
for k in final_table_us.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"cpc":k[1:]})

## 合併

In [59]:
# concat de cn
df_c = pd.concat([final_table_de,final_table_cn], join='outer')
df_c2 = pd.concat([df_c,final_table_us], join='outer')
df_c2 = df_c2.fillna(0)
#reset index
t = list(df_c2.sort_index(axis=1).columns)
df_c2 = df_c2.reindex(columns=[t[-1]]+t[:-1])
df_c2

cpc_n,company,A01,A21,A22,A23,A24,A41,A42,A43,A44,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,ABB Schweiz AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
1,ACO Severin Ahlmann GmbH and Co KG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
2,ALBA GROUP PLC & Co KG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,AT&S Austria Technologie and Systemtechnik AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.0,0.0,0.0,0.0,102.0,0.0,0.0,0.0
4,AU Optronics Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5582,incNETWORKS Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0
5583,mSignia Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
5584,nChain Holdings Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0
5585,u-blox AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,27.0,0.0,1.0,0.0,0.0


In [60]:
# 合併重複的公司
df_c2 = df_c2.groupby(df_c2.company, axis=0).sum().reset_index()
df_c2

cpc_n,company,A01,A21,A22,A23,A24,A41,A42,A43,A44,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,#glowontech,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,(c-Dot),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0
2,01 Bulletin Laboratory Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
3,01 COMMUNIQUE LABORATORY Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0
4,0Chain LLC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13364,incNETWORKS Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0
13365,mSignia Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
13366,nChain Holdings Ltd,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,138.0,0.0,0.0,0.0,0.0
13367,u-blox AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,84.0,0.0,2.0,0.0,0.0


In [61]:
df_c2.to_csv('company_cpc_all.csv',index=0)

In [62]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.company_cpc_all

In [63]:
# title
conn_db_collection.insert_one({"company":df_c2.columns.tolist()[0],"cpc":df_c2.columns.tolist()[1:]})

In [64]:
# 內容
for k in df_c2.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"cpc":k[1:]})